# RAG Based on Hybrid Search (Lexical + Semantic)


## Setting
 - Auto Reload
 - path for utils

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
module_path = "../../.."
sys.path.append(os.path.abspath(module_path))

## 1. Bedrock Client 생성

In [3]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

### ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----
- os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
- os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
- os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
- os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."

In [4]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models(verbose=False))

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Claude-V2-1': 'anthropic.claude-v2:1',
 'Claude-V3-Haiku': 'anthropic.claude-3-haiku-20240307-v1:0',
 'Claude-V3-Sonnet': 'anthropic.claude-3-sonnet-20240229-v1:0',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'amazon.titan-text-express-v1',
 'Titan-Text-G1-Light': 'amazon.titan-text-lite-v1'}


## 2. Titan Embedding 및 LLM 인 Claude-v2 모델 로딩

### LLM 로딩 (Claude-v2)

In [5]:
from langchain_community.chat_models import BedrockChat
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [6]:
llm_text = BedrockChat(
    model_id=bedrock_info.get_model_id(model_name="Claude-V3-Haiku"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens": 1024,
        "stop_sequences": ["\n\nHuman"],
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)
llm_text

BedrockChat(client=<botocore.client.BedrockRuntime object at 0x7f70ac1f98a0>, model_id='anthropic.claude-3-haiku-20240307-v1:0', model_kwargs={'max_tokens': 1024, 'stop_sequences': ['\n\nHuman']}, streaming=True, callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7f70959ce680>])

### Embedding 모델 선택

In [7]:
from utils.rag import KoSimCSERobertaContentHandler, SagemakerEndpointEmbeddingsJumpStart

In [8]:
def get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name=None):

    if is_bedrock_embeddings:
        # We will be using the Titan Embeddings Model to generate our Embeddings.
        from langchain.embeddings import BedrockEmbeddings
        llm_emb = BedrockEmbeddings(
            client=boto3_bedrock,
            model_id=bedrock_info.get_model_id(
                model_name="Titan-Embeddings-G1"
            )
        )
        print("Bedrock Embeddings Model Loaded")

    elif is_KoSimCSERobert:
        LLMEmbHandler = KoSimCSERobertaContentHandler()
        endpoint_name_emb = endpont_name
        llm_emb = SagemakerEndpointEmbeddingsJumpStart(
            endpoint_name=endpoint_name_emb,
            region_name=aws_region,
            content_handler=LLMEmbHandler,
        )
        print("KoSimCSERobert Embeddings Model Loaded")
    else:
        llm_emb = None
        print("No Embedding Model Selected")

    return llm_emb

#### [중요] is_KoSimCSERobert == True 일시에 endpoint_name 을 꼭 넣어 주세요.

In [9]:
is_bedrock_embeddings = True
is_KoSimCSERobert = False
aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

##############################
# Parameters for is_KoSimCSERobert
##############################
if is_KoSimCSERobert: endpont_name = "<endpoint-name>"
else: endpont_name = None
##############################

llm_emb = get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name)   

Bedrock Embeddings Model Loaded


## 3. LangChainOpenSearch VectorStore 정의
### 선수 조건
- 01_preprocess_docs/02_load_docs_opensearch.ipynb를 통해서 OpenSearch Index 가 생성이 되어 있어야 합니다.
#### [중요] 아래에 aws parameter store 에 아래 인증정보가 먼저 입력되어 있어야 합니다.
- 01_preprocess_docs/01_parameter_store_example.ipynb 참고

In [10]:
import boto3
from utils.ssm import parameter_store

In [11]:
region=boto3.Session().region_name
pm = parameter_store(region)

In [12]:
opensearch_domain_endpoint = pm.get_params(
    key="opensearch_domain_endpoint",
    enc=False
)

opensearch_user_id = pm.get_params(
    key="opensearch_user_id",
    enc=False
)

opensearch_user_password = pm.get_params(
    key="opensearch_user_password",
    enc=True
)

In [13]:
opensearch_domain_endpoint = opensearch_domain_endpoint
rag_user_name = opensearch_user_id
rag_user_password = opensearch_user_password

http_auth = (rag_user_name, rag_user_password) # Master username, Master password

### Index 이름 셋팅
- 이전 노트북 01_preprocess_docs/02_load_docs_opensearch.ipynb를 통해서 생성된 OpenSearch Index name 입력

In [14]:
index_name = opensearch_user_password = pm.get_params(
    key="opensearch_index_name",
    enc=True
)

print (f'index_name: {index_name}')

index_name: v01-genai-poc-parent-doc-retriever


### OpenSearch Client 생성

In [15]:
from utils.opensearch import opensearch_utils

In [16]:
os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

## 4. Retriever based on Hybrid Search 정의
- LangChain에서 제공하는 **BaseRetriever** 클래스를 상속받아 **Custom Retriever**를 정의 할 수 있습니다.
- 본 샘플코드 에서는 **Hybrid Search based Retriever**를 **정의**합니다. 

OpenSearch Hybrid 는 아래와 같은 방식으로 작동합니다.
- (1) Sematic serch를 통해 각 document별 relevant score 산출
- (2) Lexical search를 통해 각 document별 relevant score 산출
- (3-1) Rank-fusion 방식이 "simple weighted" 일 경우
    - 산출된 score에 대한 normalization 수행
    - 전체 결과에서 가장 높은 스코어는 표준화 과정을 통하여 스코어가 1.0 이 됨.
- (3-2) Rank-fusion 방식이 "Reciprocal Rank Fusion (RRF)" 일 경우
    - Paper: https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf
    - Desc: https://medium.com/@sowmiyajaganathan/hybrid-search-with-re-ranking-ff120c8a426d
    - **RRF의 경우 score가 아닌 ranking 정보를 활용, 때문에 score normalization이 필요 없음**
    - ![rrf.png](../../../10_advanced_question_answering/img/rrf.png)

RRF는 langchain에서 "Ensemble Retriever" 이름으로 api를 제공합니다. 
- https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble

In [17]:
from utils.rag import OpenSearchHybridSearchRetriever

- 필터 설정 예시
- filter=[ <BR>
    　{"term": {"metadata.[**your_metadata_attribute_name**]": "**your first keyword**"}}, <BR>
    　{"term": {"metadata.[**your_metadata_attribute_name**]": "**your second keyword**"}},<BR>
]

In [18]:
opensearch_hybrid_retriever = OpenSearchHybridSearchRetriever(
    # necessary
    os_client=os_client,
    index_name=index_name,
    llm_emb=llm_emb,
    llm_text=llm_text,

    # option for lexical
    minimum_should_match=0,
    filter=[],

    # option for rank fusion
    fusion_algorithm="RRF", # ["RRF", "simple_weighted"], rank fusion 방식 정의
    ensemble_weights=[.51, .49], # [for lexical, for semantic], Lexical, Semantic search 결과에 대한 최종 반영 비율 정의

    # option for async search
    async_mode=True,

    # option for output
    k=5, # 최종 Document 수 정의
    verbose=True,
)

### Retrieval example
- default search

In [19]:
from utils.rag import show_context_used

In [20]:
query = "중지된 경우 이체"
#query = "vidio max size?"

In [21]:
%%time
search_hybrid_result = opensearch_hybrid_retriever.get_relevant_documents(query)

print("\n==========  Results  ==========\n")
print(f'1. question: {query}')
print (f'2. # documents: {len(search_hybrid_result)}')
print("3. Documents: \n")

show_context_used(search_hybrid_result)

##############################
async_mode
##############################
True
##############################
reranker
##############################
False
##############################
rag_fusion
##############################
False
##############################
HyDE
##############################
False
##############################
parent_document
##############################
False
##############################
complex_document
##############################
False
##############################
similar_docs_semantic
##############################

Score: 1.0
['Korea. Samsung Care+ for Business에 가입된 단말의 수리는 삼성전자 서비스센터 ( www.samsungsvc.co.kr ) 에서 제공합니다. 지역별 서비스 센터 위치는 삼성전자 서비스센터 웹사이트를 통해 확인하실 수 있습니다. 서비스 운영 시간 월 ~ 금 : 09:00 ~ 19:00, 토요일 : 09:00 ~ 13:00 (일요일 · 공휴일 : 휴무) 혼잡 시간대 - 월/금요일 09시 ~ 11시 화/수/목요일 12시 ~ 14시 토요일 10시 ~ 13시 노트 - ※ 월요일/연휴 다음 날은 방문객 증가로 혼잡이 예상 되오니 다른 날 방문을 요청 드립니다. 노트 - ※ 방문객 증가 시 업무가 일찍 마감되어 당일 수리가 어려울 수 있는 점 양해 부탁드립니다. 아울러, 평일은 제품 점검 시간을 고려하여 운영시간 종료 최소 1시간 전까지 방

- update parameters

In [22]:
opensearch_hybrid_retriever.update_search_params(
    k=10,
    minimum_should_match=0,
    filter=[],
    #filter=[
    #    {"term": {"metadata.seq_num": "64"}},
    #],
    async_mode=True
)

In [23]:
query = "중지된 경우 이체"
#query = "vidio max size?"
search_hybrid_result = opensearch_hybrid_retriever.get_relevant_documents(query)

print("\n==========  Results  ==========\n")
print(f'1. question: {query}')
print(f'2. # documents: {len(search_hybrid_result)}')
print("3. Documents: \n")

show_context_used(search_hybrid_result)

##############################
async_mode
##############################
True
##############################
reranker
##############################
False
##############################
rag_fusion
##############################
False
##############################
HyDE
##############################
False
##############################
parent_document
##############################
False
##############################
complex_document
##############################
False
##############################
similar_docs_semantic
##############################

Score: 1.0
['Korea. Samsung Care+ for Business에 가입된 단말의 수리는 삼성전자 서비스센터 ( www.samsungsvc.co.kr ) 에서 제공합니다. 지역별 서비스 센터 위치는 삼성전자 서비스센터 웹사이트를 통해 확인하실 수 있습니다. 서비스 운영 시간 월 ~ 금 : 09:00 ~ 19:00, 토요일 : 09:00 ~ 13:00 (일요일 · 공휴일 : 휴무) 혼잡 시간대 - 월/금요일 09시 ~ 11시 화/수/목요일 12시 ~ 14시 토요일 10시 ~ 13시 노트 - ※ 월요일/연휴 다음 날은 방문객 증가로 혼잡이 예상 되오니 다른 날 방문을 요청 드립니다. 노트 - ※ 방문객 증가 시 업무가 일찍 마감되어 당일 수리가 어려울 수 있는 점 양해 부탁드립니다. 아울러, 평일은 제품 점검 시간을 고려하여 운영시간 종료 최소 1시간 전까지 방

## 5. RAG using RetrievalQA powered by LangChain

In [26]:
from utils.rag import prompt_repo

### Prompting
- [TIP] Prompt의 instruction의 경우 한글보다 영어로 했을 때 더 좋은 결과를 얻을 수 있습니다.

In [27]:
system_prompt = prompt_repo.get_system_prompt()
pprint (system_prompt)

('\n'
 '                        You are a master answer bot designed to answer '
 "user's questions.\n"
 "                        I'm going to give you contexts which consist of "
 'texts, tables and images.\n'
 "                        Read the contexts carefully, because I'm going to "
 'ask you a question about it.\n'
 '                        ')


### Update Search Params (Optional)

In [28]:
from utils.rag import qa_chain
from langchain.schema.output_parser import StrOutputParser

In [32]:
opensearch_hybrid_retriever.update_search_params(
    k=5,
    minimum_should_match=3,
    #filter=[
    #    {"term": {"metadata.project": "KS"}},
    #],
    filter=[],
    async_mode=True
)

### Request

In [33]:
qa = qa_chain(
    llm_text=llm_text,
    retriever=opensearch_hybrid_retriever,
    system_prompt=system_prompt,
    return_context=True,
    verbose=False
)

In [34]:
query = "중지된 경우 이체"
#query = "vidio max size?"

response, contexts = qa.invoke(
    query = query
)

show_context_used(contexts)

##############################
async_mode
##############################
True
##############################
reranker
##############################
False
##############################
rag_fusion
##############################
False
##############################
HyDE
##############################
False
##############################
parent_document
##############################
False
##############################
complex_document
##############################
False
##############################
similar_docs_semantic
##############################

Score: 1.0
['Korea. Samsung Care+ for Business에 가입된 단말의 수리는 삼성전자 서비스센터 ( www.samsungsvc.co.kr ) 에서 제공합니다. 지역별 서비스 센터 위치는 삼성전자 서비스센터 웹사이트를 통해 확인하실 수 있습니다. 서비스 운영 시간 월 ~ 금 : 09:00 ~ 19:00, 토요일 : 09:00 ~ 13:00 (일요일 · 공휴일 : 휴무) 혼잡 시간대 - 월/금요일 09시 ~ 11시 화/수/목요일 12시 ~ 14시 토요일 10시 ~ 13시 노트 - ※ 월요일/연휴 다음 날은 방문객 증가로 혼잡이 예상 되오니 다른 날 방문을 요청 드립니다. 노트 - ※ 방문객 증가 시 업무가 일찍 마감되어 당일 수리가 어려울 수 있는 점 양해 부탁드립니다. 아울러, 평일은 제품 점검 시간을 고려하여 운영시간 종료 최소 1시간 전까지 방

In [37]:
show_context_used(contexts)


-----------------------------------------------
1. Chunk: 669 Characters
-----------------------------------------------
Korea. Samsung Care+ for Business에 가입된 단말의 수리는 삼성전자 서비스센터 ( www.samsungsvc.co.kr ) 에서 제공합니다. 지역별 서비스
센터 위치는 삼성전자 서비스센터 웹사이트를 통해 확인하실 수 있습니다. 서비스 운영 시간 월 ~ 금 : 09:00 ~ 19:00, 토요일 : 09:00 ~ 13:00 (일요일 ·
공휴일 : 휴무) 혼잡 시간대 - 월/금요일 09시 ~ 11시 화/수/목요일 12시 ~ 14시 토요일 10시 ~ 13시 노트 - ※ 월요일/연휴 다음 날은 방문객 증가로 혼잡이
예상 되오니 다른 날 방문을 요청 드립니다. 노트 - ※ 방문객 증가 시 업무가 일찍 마감되어 당일 수리가 어려울 수 있는 점 양해 부탁드립니다. 아울러, 평일은 제품 점검 시간을
고려하여 운영시간 종료 최소 1시간 전까지 방문 바랍니다. 수리 상태 조회 기업 IT 장비 관리자의 경우 Samsungknox.com 에 로그인 후, Samsung Care+
for Business 메뉴 대시보드에서 Samsung Care+ for Business에 가입된 단말의 서비스 이력 및 수리 세부 상태를 확인할 수 있습니다. 문의처
삼성케어플러스 전용 콜센터 ☎ 1566-4590 (toll free number) , 월요일 ~ 일요일 24시간 지원 색인으로 돌아 가기
metadata:
 {'source': 'all_processed_data.json', 'seq_num': 1692, 'title': 'Korea', 'url':
'https://docs.samsungknox.com/admin/care-plus/submit-a-claim/korea', 'project': 'SCPE',
'last_updated': '2023-07-26',

In [ ]:
print("##################################")
print("query: ", query)
print("##################################")

print (colored("\n\n### Answer ###", "blue"))
print_ww(response)

print (colored("\n\n### Contexts ###", "green"))
show_context_used(contexts)